In [2]:
import os
import time
import numpy as np
import pandas as pd 
import datatable as dt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
from torch_geometric.datasets import QM7b
os.chdir("/mnt/home/seguraab/Shiu_Lab/Collabs/Multi_Omic/multi-omics/GCN")
import simple_GCN

11.1
cpu


Test dataset - write unittest for this below
--------------------------------------------
MoleculeNet: A Benchmark for Molecular Machine Learning
Zhenqin Wu et al. 2018

In [11]:
dataset = QM7b(root="/tmp/Cora")
edge_index = dataset.data.edge_index
edges_raw = dataset.data.edge_index.numpy()
edges = [(x, y) for x, y in zip(edges_raw[0, :], edges_raw[1, :])]
labels = dataset.data.y#.numpy()
print(dataset)
print(edge_index)
print(edges_raw)
print(edges[:5])
print(labels) # regression task

QM7b(7211)
tensor([[ 0,  0,  0,  ..., 14, 14, 14],
        [ 0,  1,  2,  ..., 12, 13, 14]])
[[ 0  0  0 ... 14 14 14]
 [ 0  1  2 ... 12 13 14]]
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]
tensor([[-4.2093e+02,  3.9695e+01,  6.2184e-01,  ...,  1.0870e+00,
          2.5346e+00,  2.4322e+00],
        [-7.1842e+02,  2.2622e+01,  6.2833e-01,  ...,  1.1186e+00,
          4.3320e+00,  4.4450e+00],
        [-5.7002e+02,  2.2941e+01,  7.0906e-01,  ...,  1.9456e+00,
          4.1733e+00,  3.9231e+00],
        ...,
        [-1.3301e+03,  1.0801e+01,  8.7788e-01,  ...,  7.0850e-01,
          1.2768e+01,  1.3187e+01],
        [-1.3555e+03,  1.0697e+01,  6.0990e-01,  ...,  6.4030e-01,
          1.3711e+01,  1.3445e+01],
        [-1.4944e+03,  1.0724e+01,  7.5581e-01,  ...,  6.0450e-01,
          1.3670e+01,  1.4056e+01]])


In [12]:
data = dataset[0]
print(data)

Data(edge_index=[2, 25], edge_attr=[25], y=[1, 14], num_nodes=5)


Genotype data analysis below this
---------------------------------

In [ ]:
# https://stackoverflow.com/questions/48152674/how-to-check-if-pytorch-is-using-the-gpu
torch.cuda.is_available()
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.device_count()
torch.cuda.get_device_name(0)
print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB') # Memory
print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB') # Memory

In [14]:
os.getcwd()

'/mnt/ufs18/home-056/seguraab/Shiu_Lab/Collabs/Multi_Omic'

In [4]:
#/mnt/home/seguraab/Shiu_Lab/Collabs/Multi_Omic/SNP_binary_matrix_383_accessions_drop_all_zero_MAF_larger_than_0.05_converted.csv
#/mnt/home/seguraab/Shiu_Lab/Collabs/Multi_Omic/Phenotype_value_383_common_accessions_2017_Grimm.csv
geno_data="SNP_binary_matrix_383_accessions_drop_all_zero_MAF_larger_than_0.05_converted.csv"
pheno_data="Phenotype_value_383_common_accessions_2017_Grimm.csv"

geno = dt.fread(geno_data) # read in genotype data
geno = geno.to_pandas() # convert dataframe to pandas dataframe
geno = geno.set_index(geno.columns[0], drop=True) # set index to sample ID
geno = geno.sort_values(by=geno.columns[0], axis=0) # sort values by sample ID
geno_sub = geno.iloc[:,0:1000]
features = geno_sub.columns # columns as features

pheno = pd.read_csv(pheno_data, index_col=0) # read in phenotype data
label = pheno.FT10_mean # flowering time as label

# Split geno and pheno into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(geno_sub, label, test_size=64)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train.values.astype(np.float32))#, device = "cuda:1") #0
X_test = torch.tensor(X_test.values.astype(np.float32))#, device = "cuda:1")
y_train = torch.tensor(y_train.values.astype(np.float32))#, device = "cuda:1") #0 2 
y_test = torch.tensor(y_test.values.astype(np.float32))#, device = "cuda:1") #3
    
#X_train = X_train.to(device="cpu")
#X_test = X_test.to(device="cpu")
#y_train = y_train.to(device="cpu")
#y_test = y_test.to(device="cpu")

#train_tensor = data.TensorDataset(X_train, y_train) # Tensor training dataset to feed
#train_loader = data.DataLoader(dataset = train_tensor, batch_size = 10, shuffle = True, pin_memory = False) # data loader to feed training data to network
#test_tensor = data.TensorDataset(X_test, y_test)
#test_loader = data.DataLoader(dataset=test_tensor, shuffle = True, pin_memory = False)

MemoryError: Unable to allocate memory of size 3627113011[errno 12] Cannot allocate memory

In [ ]:
# Initial attempt to compute adjacency matrix
from scipy import spatial
# Given that cos_sim(u, v) = dot(u, v) / (norm(u) * norm(v))
#                          = dot(u / norm(u), v / norm(v))
# We fist normalize the rows, before computing their dot products via transposition:
X_train_norm = F.normalize(X_train)
res = torch.mm(X_train_norm, X_train_norm.transpose(0,1)) # idk if this is right
print(res)

### Source:
# https://stackoverflow.com/questions/50411191/how-to-compute-the-cosine-similarity-in-pytorch-for-all-rows-in-a-matrix-with-re
# https://github.com/pytorch/pytorch/issues/11202

In [ ]:
# vs
from sklearn.metrics.pairwise import cosine_similarity
t_geno_sub = geno_sub.transpose()
adj = cosine_similarity(t_geno_sub)
adj.shape

In [ ]:
# vs
'''
MOGONET Method for Computing Adjacency Matrix
1. Calculate adjacency matrix parameter (cal_adj_mat_parameter)
    a. Compute cosine similarity matrix (cosine_distance_torch)
2. Generate adjacency matrix tensor (gen_adj_mat_tensor)
    a. Compute cosine similarity matrix (cosine_distance_torch)
    b. Generate graph from distance matrix (graph_from_dist_torch)
    c. Compute adj 1-dist ?? what's this
    d. Multiply graph by adj
    e. Transpose adj
    f. Compute identity matrix
    g. Compute adjacency matrix (not equation 2 in paper)
3. Generate test adjacency matrix tensor (gen_test_adj_mat_tensor)
'''
cuda = True if torch.cuda.is_available() else False

def cosine_distance_torch(x1, x2=None, eps=1e-8):
    x2 = x1 if x2 is None else x2
    w1 = x1.norm(p=2, dim=1, keepdim=True)
    w2 = w1 if x2 is x1 else x2.norm(p=2, dim=1, keepdim=True)
    return 1 - torch.mm(x1, x2.t()) / (w1 * w2.t()).clamp(min=eps)

dist = cosine_distance_torch(X_train)
print(dist)

def cal_adj_mat_parameter(edge_per_node, data, metric="cosine"):
    assert metric == "cosine", "Only cosine distance implemented"
    dist = cosine_distance_torch(data, data)
    parameter = torch.sort(dist.reshape(-1,)).values[edge_per_node*data.shape[0]]
    return np.asscalar(parameter.data.cpu().numpy())

def graph_from_dist_tensor(dist, parameter, self_dist=True):
    if self_dist:
        assert dist.shape[0]==dist.shape[1], "Input is not pairwise dist matrix"
    g = (dist <= parameter).float() # binary 0s and 1s for False/True
    if self_dist:
        diag_idx = np.diag_indices(g.shape[0])
        g[diag_idx[0], diag_idx[1]] = 0
    return g

def to_sparse(x):
    x_typename = torch.typename(x).split('.')[-1]
    sparse_tensortype = getattr(torch.sparse, x_typename)
    indices = torch.nonzero(x)
    if len(indices.shape) == 0:  # if all elements are zeros
        return sparse_tensortype(*x.shape)
    indices = indices.t()
    values = x[tuple(indices[i] for i in range(indices.shape[0]))]
    return sparse_tensortype(indices, values, x.size())

def gen_adj_mat_tensor(data, parameter, metric="cosine"):
    assert metric == "cosine", "Only cosine distance implemented"
    dist = cosine_distance_torch(data, data)
    g = graph_from_dist_tensor(dist, parameter, self_dist=True)
    if metric == "cosine":
        adj = 1-dist
    else:
        raise NotImplementedError
    adj = adj*g 
    adj_T = adj.transpose(0,1)
    I = torch.eye(adj.shape[0])
    if cuda:
        I = I.cuda('cuda:0')
    adj = adj + adj_T*(adj_T > adj).float() - adj*(adj_T > adj).float()
    adj = F.normalize(adj + I, p=1)
    adj = to_sparse(adj)   
    return adj

adj_parameter = 2 # edge_per_node
adj_parameter_adaptive = cal_adj_mat_parameter(adj_parameter, X_train, "cosine")
print(adj_parameter_adaptive) # this must be the threshold ϵ the paper talks about
adj_train = gen_adj_mat_tensor(X_train, adj_parameter_adaptive, "cosine")
adj_train

In [ ]:
# vs
cos = nn.CosineSimilarity()
output = cos(X_train.t(), X_train.t())
output

# vs
output2 = F.cosine_similarity(X_train.t(), X_train.t())
output2 # same as output

# vs
from sklearn.metrics.pairwise import cosine_similarity
adj_mat = cosine_similarity(X_train.t())

In [ ]:
# Define the network
net = simple_GCN.GCN(in_channels = X_train.size()[1], out_channels = [X_train.size()[1], 500, X_train.size()[0]]).to(device="cuda:4")
net = net.to(device="cuda:4")
print(net)

optimizer = torch.optim.SGD(net.parameters(), lr=0.2) # optimizer (variant of gradient descent)
loss_func = torch.nn.MSELoss()  # mean squared loss function for regression

In [18]:
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

NameError: name 'device' is not defined

In [17]:
print(torch.version.cuda)

11.1


In [ ]:
start = time.time()
# Train the network
print("Training... ")
for epoch in range(500):
    current_loss = 0.0
    print(f"\nTest: Epoch {epoch}")
    for input, batch in enumerate(train_loader, 0):
            optimizer.zero_grad()           # clear gradients for next train
            inputs, target = batch
            print("inputs", inputs)
            print("target", target)
            prediction = net(inputs) #, output2)             # prediction based on input x
            loss = loss_func(prediction, target) # must be (1. nn output, 2. target)
            loss.backward()                 # backpropagation, compute gradients
            optimizer.step()                # apply gradients

            # print statistics
            current_loss += loss.item()
            print('Loss after mini-batch %5d: %.3f' % (input + 1, current_loss / 500))
            current_loss = 0.0
    
end = time.time()   
elapsed = end - start
print("Training complete; Elapsed time: ", elapsed)

In [ ]:
# Evaluate trained model on test set
net.eval()
_, pred = net(test_tensor).max(dim=1) 
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))